In [2]:
!pip install openai

from openai import OpenAI
import networkx as nx

import numpy as np
from scipy import stats as sps
from matplotlib import pyplot as plt
import pandas as pd

Defaulting to user installation because normal site-packages is not writeable


In [3]:
import pickle
with open("data/Moviegroups.pkl", "rb") as f:
    movie_group = pickle.load(f)

In [4]:
T = 20

In [5]:
shuffled_time = np.random.RandomState(seed=42).permutation(T)
shuffled_time

array([ 0, 17, 15,  1,  8,  5, 11,  3, 18, 16, 13,  2,  9, 19,  4, 12,  7,
       10, 14,  6])

In [6]:
real_class = 0

In [7]:
N = 12

In [8]:
network = np.array([[0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1],
 [0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1],
 [1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1],
 [0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1],
 [1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1],
 [1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0],
 [0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0],
 [0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1],
 [1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1],
 [0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0],
 [0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0]])
weights = np.array([[0.333, 0.333, 0.334] for i in range(N)])

In [9]:
edges = [[] for i in range(N)]
for i in range(N):
    for j in range(i + 1, N):
        if (network[i][j] == 1):
            edges[i].append(j)
            edges[j].append(i)
    edges[i].append(i)
edges

[[3, 5, 6, 9, 0],
 [7, 8, 10, 11, 1],
 [4, 7, 9, 10, 11, 2],
 [0, 4, 6, 8, 11, 3],
 [2, 3, 5, 8, 11, 4],
 [0, 4, 6, 7, 9, 10, 11, 5],
 [0, 3, 5, 8, 6],
 [1, 2, 5, 7],
 [1, 3, 4, 6, 11, 8],
 [0, 2, 5, 10, 11, 9],
 [1, 2, 5, 9, 10],
 [1, 2, 3, 4, 5, 8, 9, 11]]

In [10]:
with open("data/Likelihoods", "rb") as f:
    id_likelihoods = np.load(f)

In [11]:
EPS = 0.011
def normalize(p, eps = EPS):
    p = np.array(p, dtype=float) + eps
    p = p / p.sum()
    return p
def normalize_big(p, eps = EPS):
    p = np.array(p, dtype=float) + eps
    p = p / p.sum(axis=1, keepdims=True)
    return p

In [16]:
T_max=15

In [17]:
def compute_ans(num_samples):
    ans = np.zeros((num_samples, T_max, N, 3))
    for sample in range(num_samples):
        prev_guess = [np.array([0.333, 0.333, 0.334]) for i in range(N)]
        for id_t in range(T_max):
            t = shuffled_time[id_t]
            for k in range(N):
                clue_id = movie_group[real_class][k][t][0]
                product = 1
                for l in edges[k]:
                    product *= prev_guess[l]
                sum_weights = (len(edges[k]))
                product = (product ** (1 / sum_weights))
                ans[sample][id_t][k] = normalize(normalize(product) * (id_likelihoods[sample][clue_id]))
            prev_guess = ans[sample][id_t]
    return ans

In [18]:
num_samples=10

In [19]:
ans = compute_ans(num_samples)

In [20]:
with open("data/NBSL_ans", "wb") as f:
    np.save(f, ans)